In [ ]:
# Standard Dependencies
import os
import cv2
import random
import numpy as np
from matplotlib import pyplot as plt
import uuid

# Tensorflow Dependencies
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

In [ ]:
# Avoiding OOM Errors for GPU
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
# Folder Structure
Pos_Path = os.path.join("data", "positive")
Neg_Path = os.path.join("data", "negative")
Anc_Path = os.path.join("data", "anchor")

In [ ]:
os.makedirs(Pos_Path)
os.makedirs(Neg_Path)
os.makedirs(Anc_Path)

 # Collecting & Processing Data

In [ ]:
# Uncompress Tar GZ Dataset
!tar -xf lfw.tgz

In [ ]:
# Movw LFW data to data/neg
for directory in os.listdir("lfw"):
    for file in os.listdir(os.path.join("lfw", directory)):
        EX_PATH = os.path.join("lfw", directory, file)
        NEW_PATH = os.path.join(Neg_Path, file)
        os.replace(EX_PATH, NEW_PATH)

In [ ]:
# Connect Webcam
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Resize Frame
    frame = frame[120:120+250, 250: 250+250, :]
    
    # Collect Anchors
    if cv2.waitKey(1) & 0XFF == ord("a"):
        # Creating Unique filepath
        imgname = os.path.join(Anc_Path, "{}.jpg".format(uuid.uuid1()))
        cv2.imwrite(imgname, frame)
    
    # Collect Positives
    if cv2.waitKey(1) & 0XFF == ord("p"):
        # Creating Unique filepath
        imgname = os.path.join(Pos_Path, "{}.jpg".format(uuid.uuid1()))
        cv2.imwrite(imgname, frame)

    # Show image back to screen
    cv2.imshow("Image Collection", frame)
    
    if cv2.waitKey(1) & 0XFF == ord("q"):
        break
# Release webcam        
cap.release()
# Close image
cv2.destroyAllWindows()

# Loading and Preprocessing Data

In [ ]:
anchor = tf.data.Dataset.list_files(Anc_Path+"\*.jpg").take(300)
positive = tf.data.Dataset.list_files(Pos_Path+"\*.jpg").take(300)
negative = tf.data.Dataset.list_files(Neg_Path+"\*.jpg").take(300)

In [ ]:
# Preprocessing
def preprocess(file_path):
    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(byte_img)
    img = tf.image.resize(img, (100, 100))
    img = img /255.0
    return img    

In [ ]:
# Creating Labelled Dataset
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

In [ ]:
samples = data.as_numpy_iterator()

In [ ]:
def preprocess_twins(input_img, validation_img, label):
    return(preprocess(input_img), preprocess(validation_img), label)

In [ ]:
# Dataloader Pipeline
data = data.map(preprocess_twins)
data = data.cache()
data = data.shuffle(buffer_size=1024)

# Training Partition 
train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

In [ ]:
# Testing Partition
test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

# Building the Model

In [ ]:
# Embedding Layer
def make_embedding():
    inp = Input(shape=(100, 100, 3), name="input_image")
    
    # First Block
    c1 = Conv2D(64, (10, 10), activation="relu")(inp)
    m1 = MaxPooling2D(64, (2, 2), padding="same")(c1)
    
    #Second Block
    c2 = Conv2D(128, (7, 7), activation="relu")(m1)
    m2 = MaxPooling2D(64, (2, 2), padding="same")(c2)
    
    #Third Block
    c3 = Conv2D(128, (4, 4), activation="relu")(m2)
    m3 = MaxPooling2D(64, (2, 2), padding="same")(c2)
    
    # Final Embedding Layer
    c4 = Conv2D(256, (4, 4), activation="relu")(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation="sigmoid")(f1)
    
    return Model(inputs=[inp], outputs=[d1], name="embedding")

In [ ]:
embedding = make_embedding()

In [ ]:
embedding.summary()

In [ ]:
# Siamese L1 Distance Class
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__()
    
    # Similarity Clarification
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [ ]:
l1 = L1Dist()

In [ ]:
# Siamese Model
def make_siamese_model():
    
    # Anchor Image
    input_image = Input(name="input_img", shape=(100, 100, 3))
    
    # Validation Image
    validation_image = Input(name="validation_img", shape=(100, 100, 3))
    
    # Siamese distance components
    siamese_layer = L1Dist()
    siamese_layer._name = "distance"
    distances = siamese_layer(embedding(input_image), embedding(validation_image))
    
    # Classification Layer
    classifier = Dense(1, activation="sigmoid")(distances)
    
    return Model(inputs=[input_image, validation_image], outputs=classifier, name="SiameseNetwork")


In [ ]:
siamese_model = make_siamese_model()

In [ ]:
siamese_model.summary()

# Training

In [ ]:
binary_cross_loss = tf.losses.BinaryCrossentropy()

In [ ]:
opt = tf.keras.optimizers.Adam(1e-4)

In [ ]:
# Checkpoints
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(opt=opt, siamese_model=siamese_model)

In [ ]:
@tf.function
def train_step(batch):
    
    with tf.GradientTape() as tape:
        # Anchor & Positive/Negative image
        X = batch[:2]
        
        # Label
        Y = batch[2]
        
        # Forward Pass
        yhat = siamese_model(X, training=True)
        # Calculate loss
        loss = binary_cross_loss(Y, yhat)
    print(loss)
        
    # Calculate Gradients
    grad = tape.gradient(loss, siamese_model.trainable_variables)
    opt.apply_gradients(zip(grad, siamese_model.trainable_variable))
    
    return loss

In [ ]:
# Training Loop
def train(data, epochs):
    for epoch in range(1, epochs+1):
        print("\n Epoch {}/{}".format(epoch, epochs))
        progbar = tf.keras.utils.Progbar(len(data))
        
        for idx, batch in enumerate(data):
            train_step(batch)
            progbar.update(idx+1)
            
        # Save Checkpoints
        if epoch % 10 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)

In [ ]:
EPOCHS = 50

In [ ]:
train(train_data, EPOCHS)

# Evaluating Model

In [ ]:
# Import metric calculations
from tensorflow.keras.metrics import Precision, Recall

In [ ]:
# Batch of Test Data
test_input, test_val, y_true = test_data.as_numpy_iterator().next()

In [ ]:
# Making Predictions
y_hat = siamese_model.predict([test_input, test_val])

# Post Processing the results
[1 if prediction > 0.5 else 0 for prediction in y_hat]

In [ ]:
y_true

In [ ]:
# Metric Object
m = Recall()

# Calculating Recall Value
m.update_state(y_true, y_hat)

# Return Recall Results
m.result().numpy()

In [ ]:
# Metric Object
m = Precision()

# Calculating Recall Value
m.update_state(y_true, y_hat)

# Return Recall Results
m.result().numpy()

# Visualization

In [ ]:
plt.figure(figsize=(18, 8))
plt.subplot(1, 2, 1)
plt.imshow(test_input[0])
plt.subplot(1, 2, 2)
plt.imshow(test_val)

# Saving the Model

In [ ]:
# Save Weights
siamese_model.save("siamesemodel.h5")

In [ ]:
# Reload the Model
model = tf.keras.models.load_model("siamesemodel.h5",
                                   custom_objects={"L1Dist":L1Dist, "BinaryCrossentropy":tf.losses.BinaryCrossentropy})